In [24]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.isidewith.com/polls/popular'
response = requests.get(url)

issues_votes = []

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    polls = soup.find_all('div', class_='poll')
    
    for poll in polls:
        # Correctly identifying the issue name
        img_div = poll.find('div', class_='img')
        if img_div:
            p_tag = img_div.find('p')
            if p_tag:
                span_tag = p_tag.find('span')
                if span_tag:
                    issue_name = span_tag.text.strip()

                    # Adjusting logic for finding vote count
                    count_div = poll.find('div', class_='count')
                    if count_div:
                        vote_text = count_div.text.strip().split()[0].replace(',', '')
                        vote_count = int(vote_text) if vote_text.isdigit() else 0
                        
                        # Appending issue name and vote count to the list
                        issues_votes.append((issue_name, vote_count))

    # Sorting and extracting the top 100 issues based on vote count
    issues_votes.sort(key=lambda x: x[1], reverse=True)
    top_issues = issues_votes[:50]

    iteration = 1
    for issue, votes in top_issues:
        print(f"Issue {iteration}: {issue}, Votes: {votes}")
        iteration = iteration + 1
else:
    print(f"Failed to fetch webpage: Status code {response.status_code}")



Issue 1: Abortion, Votes: 36417718
Issue 2: Gun Control, Votes: 36403324
Issue 3: Immigration Healthcare, Votes: 35080040
Issue 4: Obamacare, Votes: 35045122
Issue 5: Gay Marriage, Votes: 34694807
Issue 6: Marijuana, Votes: 33666167
Issue 7: Minimum Wage, Votes: 27222318
Issue 8: Terrorism, Votes: 26668564
Issue 9: Common Core, Votes: 25008043
Issue 10: Climate Change, Votes: 24709239
Issue 11: Student Loans, Votes: 24568053
Issue 12: Oil Drilling, Votes: 21665229
Issue 13: Voter Fraud, Votes: 19317014
Issue 14: Immigration, Votes: 17960706
Issue 15: Death Penalty, Votes: 16815021
Issue 16: Planned Parenthood Funding, Votes: 16750929
Issue 17: Drug Policy, Votes: 16652861
Issue 18: Border Security, Votes: 16106925
Issue 19: Affirmative Action, Votes: 15442546
Issue 20: Immigrant Laborers, Votes: 15275558
Issue 21: Fracking, Votes: 14694486
Issue 22: Equal Pay, Votes: 14456736
Issue 23: Government Mandates, Votes: 14442932
Issue 24: In-State Tuition, Votes: 14062445
Issue 25: Government

In [25]:
# Extract just the issue names from each tuple
issues = [issue[0] for issue in top_issues]

# Print out the list to verify
print(issues)

['Abortion', 'Gun Control', 'Immigration Healthcare', 'Obamacare', 'Gay Marriage', 'Marijuana', 'Minimum Wage', 'Terrorism', 'Common Core', 'Climate Change', 'Student Loans', 'Oil Drilling', 'Voter Fraud', 'Immigration', 'Death Penalty', 'Planned Parenthood Funding', 'Drug Policy', 'Border Security', 'Affirmative Action', 'Immigrant Laborers', 'Fracking', 'Equal Pay', 'Government Mandates', 'In-State Tuition', 'Government Spending', 'United Nations', 'Term Limits', 'Patriot Act', 'Solitary Confinement for Juveniles', 'Space Exploration', 'Medicaid', 'Religious Freedom Act', 'Criminal Voting Rights', 'Farm Subsidies', 'Alternative Energy', 'Israel', 'Mandatory Vaccinations', 'GMO Labels', 'Drones', 'Euthanasia', 'First Amendment', 'Immigrant Assimilation', 'No-Fly List Gun Control', 'Nuclear Energy', 'Police Body Cameras', 'Net Neutrality', 'Illegal Immigrant Detainment', 'Pension Reform', 'Gerrymandering', 'NSA Surveillance']


In [2]:
from ntscraper import Nitter

# Initialize the scraper
scraper = Nitter(log_level=1)

# Scrape a few tweets for testing
test_tweets = scraper.get_tweets("example", mode='term', number=5)

# Print the keys of the first tweet object to see what data is available
if test_tweets:
    print("Available keys in the tweet object:", test_tweets[0].keys())
else:
    print("No tweets were returned, or there was an error in scraping.")



'''import subprocess
import json
from datetime import datetime, timedelta
import pandas as pd

# Determine the timeframe for the past 30 days
date_since = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')
date_until = datetime.now().strftime('%Y-%m-%d')

# Storage for tweet data
tweets_data = []

for issue in issues:
    print(f"Fetching tweets for: {issue}")
    
    # snscrape command adjusted for timeframe and popularity (e.g., min_faves:500)
    command = f'snscrape --jsonl --max-results 5 twitter-search "{issue} since:{date_since} until:{date_until} min_faves:500 lang:en"'
    
    # Execute the command
    proc = subprocess.run(command, capture_output=True, text=True, shell=True)
    fetched_tweets = proc.stdout.strip().split('\n')
    
    for tweet_json in fetched_tweets:
        tweet = json.loads(tweet_json)
        tweets_data.append({
            'tweet': tweet.get('content'),
            'likes': tweet.get('likeCount'),
            'reposts': tweet.get('retweetCount'),
            'issue': issue,
            'url': tweet.get('url'),
            'date': tweet.get('date'),
            # Follower count is not available through snscrape, would need to use Twitter API for that
        })

# Displaying the first few entries for verification
for tweet in tweets_data[:5]:
    print(tweet)

# Convert the collected data into a pandas DataFrame
df = pd.DataFrame(tweets_data)

# Save the DataFrame to a CSV file
csv_file_path = 'collected_tweets_data.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")'''

Testing instances: 100%|██████████| 77/77 [02:00<00:00,  1.56s/it]

11-Apr-24 20:47:46 - No instance specified, using random instance https://nitter.privacydev.net


11-Apr-24 20:47:48 - Fetching error: Instance has been rate limited.Use another instance or try again later.


KeyError: 0

In [28]:
import tweepy
import pandas as pd

# Initialize Tweepy client with your bearer token
my_bearer_token = 'AAAAAAAAAAAAAAAAAAAAADG0tAEAAAAAyorYuRtjH4iMWgrLSXHucL22JhY%3DtmlXuDLwaUyPx0wkFmlOvQdUe57wANjeWsTCR8tvsSZ5bvRfOl'
client = tweepy.Client(bearer_token=my_bearer_token)

# Prepare a list to collect data
tweets_data = []

# Calculate the number of tweets to fetch per issue based on your total limit and number of issues
tweets_per_issue = 1

issue = 'abortion'

# Fetch tweets
tweets = tweepy.Paginator(client.search_recent_tweets, 
                          query=f"{issue} -is:retweet lang:en", 
                          tweet_fields=['public_metrics', 'author_id', 'created_at'],
                          max_results=1).flatten(limit=tweets_per_issue)

for tweet in tweets:
    try:
        author = client.get_user(id=tweet.author_id, user_fields=['public_metrics'])
        tweets_data.append({
            'tweet': tweet.text,
            'likes': tweet.public_metrics['like_count'],
            'reposts': tweet.public_metrics['retweet_count'],
            'follower_count': author.data.public_metrics['followers_count'],
            'issue': issue
        })
    except tweepy.TweepyException as e:
        print(f"An error occurred: {e}")

# Convert the collected data into a pandas DataFrame
df = pd.DataFrame(tweets_data)

# Save the DataFrame to a CSV file
df.to_csv('tweets_issues_data.csv', index=False)



Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.